# Setup LLM Client

In [1]:
import os
from dotenv import load_dotenv
_=load_dotenv()

In [2]:
from openai import OpenAI
client = OpenAI()
model = 'gpt-4o-mini'

In [3]:
from pprint import pprint

## Retrieval of Information from a Knowledge Base

Retrieval is another element for AI systems.

In [4]:
verbose = False # make it true for printouts
llm_temperature = 0.1 # 0->1 for creative output

### User's Question

Use any of the questions below or add a new one.

In [5]:
#user_question = 'What about the return policy?'
user_question = 'How about payment method or type?'
#user_question = 'How about shipping?'

#user_question = 'How about walking a dog in Tokyo? Be succint'
#user_question = 'How about dogs and the weather in Tokyo? Be succint'

### Knowledge data base
Create a knowledge base for retrival when requested.

In [6]:
import json

def get_knowledge(llm_question: str):
    """
    The LLM question will be used to return content.
    """
    with open("kb.json", "r") as f:
        return json.load(f)

In [7]:
from pprint import pprint
if verbose:
    print("call get_knowledge('')")
    print(type(get_knowledge('')))
    pprint(get_knowledge(''))

### OpenAI Tool Function

The mechanism to let the LLM know there exists an additional knowlege base.

In [8]:
'''
docs: https://platform.openai.com/docs/guides/function-calling
'''
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_knowledge",
            "description": "Get knowledge to answer the user's question.",
            "parameters": {
                "type": "object",
                "properties": {
                    "llm_question": {"type": "string"},
                },
                "required": ["llm_question"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

### Invoke the LLM with the User's Question

In [9]:
# This gives a context for the llm assistant
system_prompt = "You are a helpful assistant that answers questions from the knowledge base "+\
                "about our e-commerce store but you also have general knowledge."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_question},
]

completion = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
    temperature=0.0,
)

In [10]:
'''Inspect the response data'''
if verbose:
    pprint(completion.model_dump())

In [11]:
if verbose:
    pprint(completion.usage)

In [12]:
if verbose:
    pprint(completion.choices[0].message)

In [13]:
if verbose:
    if completion.choices[0].message.tool_calls:
       pprint(completion.choices[0].message.tool_calls[0].function.arguments)

In [14]:
 print('total_tokens=', completion.usage.total_tokens)

total_tokens= 109


### Add Knowledge Base to Prompt

In [15]:
# Add to prompt if the tool function was activated
if completion.choices[0].message.tool_calls:

    def call_function(name, args):
        if name == "get_knowledge":
            return get_knowledge(**args)
    
    for i, tool_call in enumerate(completion.choices[0].message.tool_calls):
        if verbose:
            print('i = ', i)
        name = tool_call.function.name
        llm_question = json.loads(tool_call.function.arguments)
        
        if verbose:
            print('llm_question =', llm_question)
    
        messages.append(completion.choices[0].message) # this needs to be here before the "tool role"

        result = call_function(name, llm_question)
        
        messages.append(
            {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
        )

In [16]:
if verbose:
    pprint(messages)

### Invoke the LLM with the User's Question and Knowlege Base

In [17]:
# Invoke with knowlege base only if the tool function was activated
if completion.choices[0].message.tool_calls:
    from pydantic import BaseModel, Field

    # Desired response format
    class Reply(BaseModel):
        answer: str = Field(description="The answer to the user's question.")
        source: int = Field(description="The record id of the answer if there is one. If a record id cannot be found set the id to zero.")

    completion = client.beta.chat.completions.parse(
        model=model,
        messages=messages,
        tools=tools,
        response_format=Reply,
        temperature=0.0,
    )

In [18]:
if verbose:
    pprint(completion.model_dump())

In [19]:
if verbose:
    pprint(completion.usage)

In [20]:
if verbose:
    pprint(completion.choices[0].message)

In [21]:
if verbose:
    pprint(completion.choices[0].message.content)

In [22]:
'''Message content is not empty.'''
if verbose:
    pprint(completion.choices[0].message.role)

In [23]:
if verbose:
    print('messages =')
    pprint(messages)
    print('')
    print('Type =\n', type(completion.model_dump()))
    print('')
    print('chat completion dump =')
    pprint(completion.model_dump())

In [24]:
print('total_tokens=', completion.usage.total_tokens)

total_tokens= 403


### Extract Structured Response

In [27]:
try:
    final_reply = completion.choices[0].message.parsed
    print('Answer:\n', final_reply.answer)
    print('')
    print('Source =', final_reply.source)
except:
    print(completion.choices[0].message.content)

Answer:
 We accept Visa, Mastercard, American Express, PayPal, and Apple Pay. All payments are processed securely through our encrypted payment system.

Source = 3
